In [6]:
import sqlite3
import pandas as pd

In [7]:
conn = sqlite3.connect(r"..\data\raw\olist.sqlite")

In [8]:
orders = pd.read_sql("SELECT * FROM orders;", conn)
customers = pd.read_sql("SELECT * FROM customers;", conn)
products = pd.read_sql("SELECT * FROM products;", conn)
order_items = pd.read_sql("SELECT * FROM order_items;", conn)


In [9]:
orders = orders.dropna(subset=["order_id", "customer_id"])

date_cols = [col for col in orders.columns if "timestamp" in col or "date" in col]
for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

order_items["price"] = order_items["price"].fillna(0)
order_items["freight_value"] = order_items["freight_value"].fillna(0)


In [10]:
print("Missing values after cleaning:")
print(orders.isnull().sum())
print(order_items.isnull().sum())

Missing values after cleaning:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


### Handling Missing Values

- **`order_approved_at` (160 missing):**  
  These orders were purchased but never approved (likely canceled or failed payment).  
  → Decision: Keep them, since they reflect real-world business cases.  

- **`order_delivered_carrier_date` (1,783 missing):**  
  The seller never handed the package to the carrier.  
  → Decision: Keep them, mark as "not delivered" in later analysis.  

- **`order_delivered_customer_date` (2,965 missing):**  
  The package never reached the customer.  
  → Decision: Keep them, as they provide useful information for analyzing delivery performance and cancellations.  

- **`order_estimated_delivery_date` (0 missing):**  
  Always filled, no action needed.  

- **`order_items` table:**  
  No missing values after cleaning.  

**Summary:**  
We did **not drop rows** with missing delivery/approval dates, since they are meaningful business events (canceled or failed deliveries).  
For `price` and `freight_value`, missing values were replaced with `0`, assuming no charge.  
This ensures the dataset remains realistic and ready for analysis without losing important information.


In [11]:
orders["order_id"] = orders["order_id"].astype(str)
customers["customer_id"] = customers["customer_id"].astype(str)
products["product_id"] = products["product_id"].astype(str)

order_items["price"] = order_items["price"].astype(float)
order_items["freight_value"] = order_items["freight_value"].astype(float)


In [12]:
orders = orders.drop_duplicates(subset="order_id")
customers = customers.drop_duplicates(subset="customer_id")
products = products.drop_duplicates(subset="product_id")
order_items = order_items.drop_duplicates(subset=["order_id", "order_item_id"])

In [13]:
orders["order_month"] = orders["order_purchase_timestamp"].dt.to_period("M")
orders["order_weekday"] = orders["order_purchase_timestamp"].dt.day_name()

order_values = (
    order_items.groupby("order_id")
    .agg(order_value=("price", "sum"), freight=("freight_value", "sum"))
    .reset_index()
)
order_values["order_total"] = order_values["order_value"] + order_values["freight"]


In [14]:
main = (
    orders
    .merge(customers, on="customer_id", how="left")
    .merge(order_values, on="order_id", how="left")
)

print("Main dataset shape:", main.shape)
print(main.head())

Main dataset shape: (99441, 17)
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-

In [15]:
orders.to_csv("../data/processed/clean_orders.csv", index=False)
customers.to_csv("../data/processed/clean_customers.csv", index=False)
products.to_csv("../data/processed/clean_products.csv", index=False)
order_items.to_csv("../data/processed/clean_order_items.csv", index=False)
main.to_csv("../data/processed/clean_main_dataset.csv", index=False)